In [0]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

import cv2

!rm *.jpg
!rm *.txt
!rm *.JPG


#Upload images to google colab
print("Upload Training Images...")
train_images = files.upload()
print("Upload Training Labels...")
train_labels = files.upload()
print("Upload Validation Images...")
validation_images = files.upload()
print("Upload Validation Labels...")
validation_labels = files.upload()
print("Upload Testing Images...")
test_images = files.upload()
print("Upload Testing Labels...")
test_labels = files.upload()


##Preprocessing of Images

#create lists of files for training, validation, and test images
train_images_list = list(train_images.keys())
validation_images_list = list(validation_images.keys())
test_images_list = list(test_images.keys())

#create lists of files for training, validation, and test labels
train_labels_list = list(train_labels.keys())
validation_labels_list = list(validation_labels.keys())
test_labels_list = list(test_labels.keys())

#get pixel array of training dataset
train_images_array = []
for image in train_images_list:
  image_pixel_array = cv2.imread(image)
  train_images_array.append(image_pixel_array)
#convert array to numpy array
train_images_nparray = np.array(train_images_array)

#get pixel array of validation dataset
validation_images_array = []
for image in validation_images_list:
  image_pixel_array = cv2.imread(image)
  validation_images_array.append(image_pixel_array)
#convert array to numpy array
validation_images_nparray = np.array(validation_images_array)

#get pixel array of test dataset
test_images_array = []
for image in test_images_list:
  image_pixel_array = cv2.imread(image)
  test_images_array.append(image_pixel_array)
#convert array to numpy array
test_images_nparray = np.array(test_images_array)

#make all pixel values between 0 and 1
train_images_nparray = train_images_nparray / 255.0
validation_images_nparray = validation_images_nparray / 255.0
test_images_nparray = test_images_nparray / 255.0

#Save all the labels in list form
train_class_list = []
for label_file in train_labels_list:
  label_file_open = open(label_file, "r")
  file_contents = label_file_open.read()
  object_class = file_contents.split('\t', 1)[0]
  if object_class == 'Coin':
    train_class_list.append(0)
  elif object_class == 'Cash':
    train_class_list.append(1)
train_class_nparray = np.asarray(train_class_list)

validation_class_list = []
for label_file in validation_labels_list:
  label_file_open = open(label_file, "r")
  file_contents = label_file_open.read()
  object_class = file_contents.split('\t', 1)[0]
  if object_class == 'Coin':
    validation_class_list.append(0)
  elif object_class == 'Cash':
    validation_class_list.append(1)
validation_class_nparray = np.asarray(validation_class_list)

test_class_list = []
for label_file in test_labels_list:
  label_file_open = open(label_file, "r")
  file_contents = label_file_open.read()
  object_class = file_contents.split('\t', 1)[0]
  if object_class == 'Coin':
    test_class_list.append(0)
  elif object_class == 'Cash':
    test_class_list.append(1)
test_class_nparray = np.asarray(test_class_list)

##Build Model1
model1 = keras.Sequential([
        keras.layers.Conv2D(64, (6,6), activation='relu', input_shape=(80,80,3), data_format='channels_last'),
        keras.layers.Conv2D(32, (6,6), activation='relu', data_format='channels_last'),
        keras.layers.Flatten(), 
        keras.layers.Dense(2, activation='softmax')
        ])  

##Compile Model1
model1.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##Train Model1
model1.fit(train_images_nparray, 
          train_class_nparray, 
          validation_data=(validation_images_nparray, validation_class_nparray),
          epochs=100,
          shuffle=True)

##Build Model2
model2 = keras.Sequential([
        keras.layers.Conv2D(32, (6,6), activation='relu', input_shape=(80,80,3), data_format='channels_last'),
        keras.layers.Conv2D(32, (6,6), activation='relu', data_format='channels_last'),
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(2, activation='softmax')
])

##Compile Model2
model2.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##Train Model2
model2.fit(train_images_nparray, 
          train_class_nparray, 
          validation_data=(validation_images_nparray, validation_class_nparray),
          epochs=100,
          shuffle=True)

##Make Predictions
predictions1 = model1.predict(test_images_nparray)
print(predictions1)
predictions2 = model2.predict(test_images_nparray)
print(predictions2)


